In [10]:
import pandas as pd
from minio import Minio
from minio.error import S3Error
from io import BytesIO
import io

# Configurações do Minio
minio_host = 'minio.ifpbapps.online'
minio_access_key = "VQdTNNVXZZLXNMRnV4RDFoQ2MtZGFSRjN3Q0Z2Z"
minio_secret_key = "pMnQ4ZkktaU9UdlRYX1c!3S1lHMkVpajU5LTRLTHl1NwbmRDVmdiWW9RaGJvZmFWSkF5YmZ"
minio_bucket = 'raios'
minio_folder = ''

# Conectar ao Minio
minio_client = Minio(minio_host, access_key=minio_access_key, secret_key=minio_secret_key, secure=True)

# Função para baixar arquivo do Minio e carregar como DataFrame
def download_and_read_csv(minio_client, bucket, file_path, separator=','):
    try:
        # Baixar o arquivo do Minio
        data = minio_client.get_object(bucket, file_path)
        
        # Ler o CSV usando pandas e especificar o separador
        df = pd.read_csv(BytesIO(data.read()), sep=separator)
        
        return df
    except S3Error as err:
        print(err)
        return None

# Lista de arquivos no Minio
files = [
    'Consolidado Raios 2023_EPB.csv',
    'Consolidado Raios 2023_ERO.csv',
    'Consolidado Raios 2023_EMT.csv',
    'Consolidado Raios 2023_EMS.csv',
    'Consolidado Raios 2023_EMR.csv',
    'Consolidado Raios 2023_EAC.csv',
    'Consolidado Raios 2023_ETO.csv',
    'Consolidado Raios 2023_ESS.csv',
    'Consolidado Raios 2023_ESE.csv'
]

# Dicionário para armazenar DataFrames
dfs = {}

# Iterar sobre os arquivos e baixar do Minio
for file in files:
    df = download_and_read_csv(minio_client, minio_bucket, file, separator=';')
    if df is not None:
        dfs[file] = df

# Agora, dfs é um dicionário onde as chaves são os nomes dos arquivos e os valores são os DataFrames correspondentes.
# Você pode acessar cada DataFrame usando o nome do arquivo como chave.

Descargas_EPB = dfs['Consolidado Raios 2023_EPB.csv']
Descargas_ERO = dfs['Consolidado Raios 2023_ERO.csv']
Descargas_ETO = dfs['Consolidado Raios 2023_ETO.csv']
Descargas_EMS = dfs['Consolidado Raios 2023_EMS.csv']
Descargas_EMR = dfs['Consolidado Raios 2023_EMR.csv']
Descargas_EMT = dfs['Consolidado Raios 2023_EMT.csv']
Descargas_EAC = dfs['Consolidado Raios 2023_EAC.csv']
Descargas_ESS = dfs['Consolidado Raios 2023_ESS.csv']
Descargas_ESE = dfs['Consolidado Raios 2023_ESE.csv']

In [11]:
# Carrega os códigos do IBGE
cod_ibge_data = minio_client.get_object(minio_bucket, f'{minio_folder}dados-ibge.xls')
cod_ibge_buffer = io.BytesIO(cod_ibge_data.read())
Cod_IBGE = pd.read_excel(cod_ibge_buffer, header=6)

# Selecionar apenas as colunas desejadas
Cod_IBGE = Cod_IBGE[['Nome_UF', 'Código Município Completo', 'Nome_Município']]

# Renomear as colunas
Cod_IBGE = Cod_IBGE.rename(columns={
    'Nome_UF': 'UF',
    'Código Município Completo': 'Cod_Municipio',
    'Nome_Município': 'Nome_Municipio'
})


WARNING *** OLE2 inconsistency: SSCS size is 0 but SSAT size is non-zero
*** No CODEPAGE record, no encoding_override: will use 'iso-8859-1'


In [12]:
# Carrega as ocorrências da ANEEL
ocorrencias_data = minio_client.get_object(minio_bucket, f'{minio_folder}Ocorrencias_ANEEL_2023.csv')
ocorrencias_buffer = io.BytesIO(ocorrencias_data.read())
Ocorrencias = pd.read_csv(ocorrencias_buffer, delimiter=',', encoding='latin-1', low_memory=False)

In [13]:
novos_nomes_colunas = ['Nome_Municipio', 'Janeiro', 'Fevereiro', 'Marco', 'Abril', 'Maio', 'Junho', 'Julho', 'Agosto', 'Setembro', 'Outubro', 'Novembro', 'Dezembro']

# Mantendo apenas as colunas da base que possuem dados de raios no sentido Nuvem-Solo
Descargas_EMR = Descargas_EMR.iloc[:, [0, 1, 4, 7, 10, 13, 16, 19, 22, 25, 28, 31, 34]]

# Excluir a primeira linha, pois a tabela só começa na terceira linha
Descargas_EMR = Descargas_EMR.drop(Descargas_EMR.index[0])

# Renomear todas as colunas para que fique de acordo com o número de raios nos meses correspondentes
Descargas_EMR.columns = novos_nomes_colunas

Descargas_EPB = Descargas_EPB.iloc[:, [0, 1, 4, 7, 10, 13, 16, 19, 22, 25, 28, 31, 34]]
Descargas_EPB = Descargas_EPB.drop(Descargas_EPB.index[0])
Descargas_EPB.columns = novos_nomes_colunas

Descargas_ERO = Descargas_ERO.iloc[:, [0, 1, 4, 7, 10, 13, 16, 19, 22, 25, 28, 31, 34]]
Descargas_ERO = Descargas_ERO.drop(Descargas_ERO.index[0])
Descargas_ERO.columns = novos_nomes_colunas

Descargas_EMS = Descargas_EMS.iloc[:, [0, 1, 4, 7, 10, 13, 16, 19, 22, 25, 28, 31, 34]]
Descargas_EMS = Descargas_EMS.drop(Descargas_EMS.index[0])
Descargas_EMS.columns = novos_nomes_colunas

Descargas_EMT = Descargas_EMT.iloc[:, [0, 1, 4, 7, 10, 13, 16, 19, 22, 25, 28, 31, 34]]
Descargas_EMT = Descargas_EMT.drop(Descargas_EMT.index[0])
Descargas_EMT.columns = novos_nomes_colunas

Descargas_EAC = Descargas_EAC.iloc[:, [0, 1, 4, 7, 10, 13, 16, 19, 22, 25, 28, 31, 34]]
Descargas_EAC = Descargas_EAC.drop(Descargas_EAC.index[0])
Descargas_EAC.columns = novos_nomes_colunas

Descargas_ETO = Descargas_ETO.iloc[:, [0, 1, 4, 7, 10, 13, 16, 19, 22, 25, 28, 31, 34]]
Descargas_ETO = Descargas_ETO.drop(Descargas_ETO.index[0])
Descargas_ETO.columns = novos_nomes_colunas

Descargas_ESS = Descargas_ESS.iloc[:, [0, 1, 4, 7, 10, 13, 16, 19, 22, 25, 28, 31, 34]]
Descargas_ESS = Descargas_ESS.drop(Descargas_ESS.index[0])
Descargas_ESS.columns = novos_nomes_colunas

Descargas_ESE = Descargas_ESE.iloc[:, [0, 1, 4, 7, 10, 13, 16, 19, 22, 25, 28, 31, 34]]
Descargas_ESE = Descargas_ESE.drop(Descargas_ESE.index[0])
Descargas_ESE.columns = novos_nomes_colunas

In [14]:
# Definindo os DataFrames IBGE para cada estado
IBGE_EPB = Cod_IBGE[Cod_IBGE['UF'] == 'Paraíba']
IBGE_ESE = Cod_IBGE[Cod_IBGE['UF'] == 'Sergipe']
IBGE_ETO = Cod_IBGE[Cod_IBGE['UF'] == 'Tocantins']
IBGE_EAC = Cod_IBGE[Cod_IBGE['UF'] == 'Acre']
IBGE_ERO = Cod_IBGE[Cod_IBGE['UF'] == 'Rondônia']
IBGE_EMT = Cod_IBGE[Cod_IBGE['UF'] == 'Mato Grosso']
IBGE_EMS = Cod_IBGE[Cod_IBGE['UF'] == 'Mato Grosso do Sul']
IBGE_ESS = Cod_IBGE[Cod_IBGE['UF'].isin(['São Paulo', 'Paraná', 'Minas Gerais'])]
IBGE_EMR = Cod_IBGE[Cod_IBGE['UF'].isin(['Minas Gerais', 'Rio de Janeiro'])]

# Mesclar cada Descargas_EXX com seu equivalente IBGE_EXX
Descargas_EPB = pd.merge(Descargas_EPB, IBGE_EPB, on='Nome_Municipio', how='inner')
Descargas_ESE = pd.merge(Descargas_ESE, IBGE_ESE, on='Nome_Municipio', how='inner')
Descargas_ETO = pd.merge(Descargas_ETO, IBGE_ETO, on='Nome_Municipio', how='inner')
Descargas_EAC = pd.merge(Descargas_EAC, IBGE_EAC, on='Nome_Municipio', how='inner')
Descargas_ERO = pd.merge(Descargas_ERO, IBGE_ERO, on='Nome_Municipio', how='inner')
Descargas_EMT = pd.merge(Descargas_EMT, IBGE_EMT, on='Nome_Municipio', how='inner')
Descargas_EMS = pd.merge(Descargas_EMS, IBGE_EMS, on='Nome_Municipio', how='inner')
Descargas_ESS = pd.merge(Descargas_ESS, IBGE_ESS, on='Nome_Municipio', how='inner')
Descargas_EMR = pd.merge(Descargas_EMR, IBGE_EMR, on='Nome_Municipio', how='inner')

# Deixando um único dataframe pra todas as descargas do grupo Energisa
Lista_Descargas = [Descargas_EPB, Descargas_ERO, Descargas_EMT, Descargas_EMS, Descargas_EMR, Descargas_EAC, Descargas_ETO, Descargas_ESS, Descargas_ESE]
Descargas_Energisa = pd.concat(Lista_Descargas, ignore_index=True)

In [15]:
Ocorrencias = Ocorrencias[['Tipo_Ocorrencia', 'Cod_Municipio']]

Ocorrencias = Ocorrencias['Cod_Municipio'].value_counts()

Ocorrencias = Ocorrencias.reset_index()
Ocorrencias.columns = ['Cod_Municipio', 'Quantidade_de_Ocorrencias']

In [16]:
# ADICIONAR COLUNA DE SOMA DOS MESES DE JANEIRO A DEZEMBRO, NO DATAFRAME INTEIRO MESMO.
# Convertendo as colunas de meses para o tipo numérico, substituindo valores não numéricos por 0
Descargas_Energisa.iloc[:, 1:13] = Descargas_Energisa.iloc[:, 1:13].map(lambda x: pd.to_numeric(x, errors='coerce')).fillna(0)

# Agora, criamos a coluna "Total" somando os valores das colunas de meses
Descargas_Energisa['Total'] = Descargas_Energisa.iloc[:, 1:13].sum(axis=1)

# Ocorrencias = pd.merge(Ocorrencias, Cod_IBGE, on='Cod_Municipio', how='inner')
Ocorrencias_Raios = pd.merge(Ocorrencias, Descargas_Energisa, on='Cod_Municipio', how='inner')


In [17]:
Ocorrencias_Raios['Municipio_Estado'] = Ocorrencias_Raios.apply(lambda row: f"{row['Nome_Municipio']} - {row['UF']}", axis=1)

In [18]:
Ocorrencias_Raios.to_csv('Ocorrencias_Raios.csv',index=False)